<a href="https://colab.research.google.com/github/APanico12/Lezione-PCTO/blob/main/Applicazione_dataset_industriale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Introduzione al problema

In questo progetto, il nostro obiettivo è costruire un **modello predittivo della qualità del prodotto** generato da una **macchina per la tostatura a camere multiple**.

La macchina è composta da **5 camere**, ognuna delle quali contiene **3 sensori di temperatura**, disposti in modo da monitorare uniformemente la distribuzione del calore. Oltre alla temperatura, vengono rilevate due caratteristiche **prima dell'inizio del processo**:  
- **L’umidità iniziale** del materiale grezzo  
- **L’altezza dello strato** di materiale immesso nella macchina

---

# 🔄 Processo di tostatura

- Il processo dura **esattamente 1 ora** per ogni batch di materiale.
- I **sensori registrano dati ogni minuto**, quindi per ogni batch abbiamo **60 osservazioni** temporali per ciascun sensore.
- Alla fine dell'ora, un **campione del prodotto tostato** viene analizzato in laboratorio e ne viene misurata la **qualità complessiva**.

---

# 📁 I dataset forniti

## 🔹 `data_X.csv`

Contiene i dati minuto per minuto raccolti durante il processo di tostatura.  
Ogni riga rappresenta **una lettura al minuto**, e include:

- 15 colonne di temperatura: `temp_0` a `temp_14`  
  (3 sensori per ciascuna delle 5 camere)
- 2 colonne statiche (costanti durante il batch ma ripetute ogni minuto):
  - `layer_height` → altezza dello strato del materiale
  - `moisture` → umidità iniziale del materiale
- `time` → timestamp del minuto

Un batch da 1 ora è quindi rappresentato da **60 righe consecutive**.

---

## 🔹 `data_Y.csv`

Contiene i **valori di qualità misurati** in laboratorio.  
Ogni riga corrisponde a un **campione orario** del prodotto in uscita, con:

- `time` → timestamp dell'ora
- `target` → qualità misurata del prodotto

L'etichetta (`target`) si riferisce al materiale processato nei **60 minuti precedenti**.

---

## 🔹 `sample_submission.csv`

Contiene l’elenco dei **timestamp** per cui è richiesta una predizione di qualità.  
Il file è utilizzato per formattare correttamente le **5808 predizioni** finali da inviare per la valutazione.

---

# 🎯 Obiettivo del modello

Dobbiamo costruire un modello che:

- A partire dai **dati dei sensori ogni minuto** (più altezza e umidità iniziali),
- Predica il valore di **qualità del prodotto** alla fine dell'ora,
- Minimizzando l'**errore assoluto medio (MAE)** rispetto alle misurazioni di laboratorio.

Ogni istanza di apprendimento corrisponde a **un batch orario**, e il modello dovrà associare le informazioni temporali (60×sensori) a **una sola etichetta finale** di qualità.

---

# 💡 Strategia suggerita

- Raggruppare ogni batch di 60 righe (1 ora) in una singola istanza
- Calcolare statistiche aggregative (media, deviazione standard, min, max) per ogni sensore
- Utilizzare queste statistiche + le variabili statiche (`layer_height`, `moisture`) come feature
- Addestrare un modello di regressione (es. Random Forest, XGBoost, o reti neurali)


In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Install dependencies as needed:
#!pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "data_X.csv"

# Load the latest version
X = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "podsyp/production-quality",
  file_path
)
Y = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "podsyp/production-quality",
  "data_Y.csv"
)
submission = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "podsyp/production-quality",
  "sample_submission.csv"
)

<ipython-input-4-207545901>:10: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  X = kagglehub.load_dataset(


100%|██████████| 30.0M/30.0M [00:00<00:00, 91.9MB/s]

Extracting zip of data_X.csv...



<ipython-input-4-207545901>:15: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  Y = kagglehub.load_dataset(
<ipython-input-4-207545901>:20: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  submission = kagglehub.load_dataset(


# 🔍 Inizio EDA (Exploratory Data Analysis)

Quando affrontiamo un problema di machine learning o analisi statistica, il primo passo fondamentale è **conoscere i dati**. L’EDA serve a:

- Capire la **struttura del dataset**
- Verificare la **qualità dei dati** (missing, formati, outlier)
- Osservare la **distribuzione delle variabili**
- Identificare pattern, correlazioni o anomalie
- Preparare le **trasformazioni e feature engineering** successive

---

## 🗂️ Data Description

Il primo passo in ogni analisi statistica è capire **che tipo di dati abbiamo a disposizione**.

Nel nostro caso, vogliamo costruire un modello in grado di prevedere la **qualità finale del prodotto** tostato, sulla base di misurazioni effettuate **minuto per minuto** durante il processo produttivo.

---

### 📐 Dimensioni dei dataset

Verifichiamo il numero di righe (osservazioni) e colonne (variabili) per ciascun file.


In [5]:
X.shape, Y.shape, submission.shape

((2103841, 18), (29184, 2), (5808, 2))

In [6]:
#info for all dataset
X.info(), Y.info(), submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2103841 entries, 0 to 2103840
Data columns (total 18 columns):
 #   Column      Dtype  
---  ------      -----  
 0   date_time   object 
 1   T_data_1_1  int64  
 2   T_data_1_2  int64  
 3   T_data_1_3  int64  
 4   T_data_2_1  int64  
 5   T_data_2_2  int64  
 6   T_data_2_3  int64  
 7   T_data_3_1  int64  
 8   T_data_3_2  int64  
 9   T_data_3_3  int64  
 10  T_data_4_1  int64  
 11  T_data_4_2  int64  
 12  T_data_4_3  int64  
 13  T_data_5_1  int64  
 14  T_data_5_2  int64  
 15  T_data_5_3  int64  
 16  H_data      float64
 17  AH_data     float64
dtypes: float64(2), int64(15), object(1)
memory usage: 288.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29184 entries, 0 to 29183
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date_time  29184 non-null  object
 1   quality    29184 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 456.1+ KB
<class

(None, None, None)

In [7]:
X.describe()[1:].T.style.background_gradient(cmap='GnBu', axis=1)

,mean,std,min,25%,50%,75%,max
T_data_1_1,250.182182,32.116609,-198.000000,229.000000,250.000000,272.000000,724.000000
T_data_1_2,250.091514,30.824510,-122.000000,229.000000,250.000000,272.000000,762.000000
T_data_1_3,250.253580,30.691253,-107.000000,229.000000,250.000000,272.000000,665.000000
T_data_2_1,349.775603,42.259606,-703.000000,328.000000,350.000000,372.000000,1302.000000
T_data_2_2,349.721759,40.678867,-958.000000,328.000000,350.000000,372.000000,1179.000000
T_data_2_3,349.828165,37.984437,-191.000000,328.000000,350.000000,372.000000,889.000000
T_data_3_1,501.161070,63.257324,-775.000000,464.000000,502.000000,538.000000,1587.000000
T_data_3_2,501.109222,63.371641,-759.000000,464.000000,502.000000,538.000000,2505.000000
T_data_3_3,501.233550,62.232944,-613.000000,464.000000,502.000000,539.000000,1319.000000
T_data_4_1,349.319616,39.305621,-514.000000,327.000000,349.000000,372.000000,1177.000000


In [8]:
submission.describe()[1:].T.style.background_gradient(cmap='GnBu', axis=1)

,mean,std,min,25%,50%,75%,max
quality,420.000000,0.000000,420.000000,420.000000,420.000000,420.000000,420.000000


In [9]:
Y.describe()[1:].T.style.background_gradient(cmap='GnBu', axis=1)

,mean,std,min,25%,50%,75%,max
quality,402.800747,46.273228,221.000000,372.000000,408.000000,439.000000,505.000000


## 🧼 Analisi dei Missing Values

L'analisi dei valori mancanti è un passo fondamentale dell'EDA. Capire **dove e quanto sono presenti i NaN** ci aiuta a:

- Determinare **quali osservazioni possiamo includere** nell'analisi
- Decidere se conviene **imputare** (riempire) i valori o **scartare** le righe/colonne
- Valutare se i NaN sono casuali o seguono un **pattern sistematico**

⚠️ Valori nulli non gestiti correttamente possono ridurre la **qualità dell’analisi** e introdurre **bias** nel modello.


In [ ]:
rows, cols = X.shape[0],X.shape[1]
total_cells = rows * cols
num_ones = int(0.2 * total_cells)

# Inizialmente tutti 0
binary_matrix = np.zeros(total_cells, dtype=int)

# Impostiamo 1 in posizioni casuali
np.random.seed(42)
binary_matrix[:num_ones] = 1
np.random.shuffle(binary_matrix)


binary_matrix = binary_matrix.reshape((rows, cols))
binary_df = pd.DataFrame(binary_matrix, columns=[f"col_{i+1}" for i in range(cols)])
plt.figure(figsize=(22,4))

sns.heatmap(binary_matrix,cmap='GnBu',cbar=False, annot=False,
             fmt='0.0f').set_title('Count of Missing Values', fontsize=18)
plt.show()


In [ ]:
plt.figure(figsize=(22,4))
sns.heatmap((X.isna().sum()).to_frame(name='').T,cmap='GnBu',cbar=False, annot=False,
             fmt='0.0f').set_title('Count of Missing Values', fontsize=18)
plt.show()

In [ ]:
plt.figure(figsize=(22,4))
sns.heatmap((submission.isna().sum()).to_frame(name='').T,cmap='GnBu',cbar=False, annot=False,
             fmt='0.0f').set_title('Count of Missing Values', fontsize=18)
plt.show()

In [ ]:
plt.figure(figsize=(22,4))
sns.heatmap((Y.isna().sum()).to_frame(name='').T,cmap='GnBu',cbar=False, annot=False,
             fmt='0.0f').set_title('Count of Missing Values Y ', fontsize=18)
plt.show()

## 📊 Analisi delle relazioni tra le variabili

L’analisi delle relazioni tra le variabili è una fase fondamentale dell’EDA (Analisi Esplorativa dei Dati), perché ci permette di:

- 🧠 **Capire quali variabili influenzano maggiormente il target**  
  Se una variabile ha una forte correlazione con la variabile `quality`, è un buon candidato come feature predittiva.

- 🧹 **Eliminare ridondanze**  
  Se due variabili sono fortemente correlate tra loro (es. `temp_1` e `temp_2`), una delle due potrebbe essere superflua.

- 🧪 **Validare ipotesi**  
  Ad esempio: “ci aspettiamo che all’aumentare dell’umidità, la qualità diminuisca” → la correlazione ci dice se è vero.

- ⚙️ **Guidare il feature engineering**  
  Possiamo decidere di combinare, trasformare o selezionare variabili in base alle relazioni osservate.

- 🚀 **Costruire modelli più semplici ed efficienti**  
  Rimuovendo le variabili inutili o poco informative, si migliora la velocità e la qualità del modello.

> In conclusione, questa fase ci aiuta a selezionare le informazioni più rilevanti, migliorare la struttura dei dati e ottimizzare le performance del nostro modello predittivo.


In [ ]:
import datetime

X["date_time"] = pd.to_datetime(X["date_time"])
X["date_hour"] = X["date_time"].apply(lambda x: x.strftime("%d-%m-%Y-%H"))

Y["date_shifted"] = pd.to_datetime(Y["date_time"]) - datetime.timedelta(hours=1) #perchè si riferisce all' ora precedente
Y["date_shifted"] = Y["date_shifted"].apply(lambda x: x.strftime("%d-%m-%Y-%H"))

training = pd.merge(X, Y[["date_shifted", "quality"]], left_on="date_hour", right_on="date_shifted", how="inner")


In [ ]:
#copare the data
training['date_hour'].nunique(),Y.shape[0]

In [ ]:
# Aggrega i dati al livello orario
training.head()

In [ ]:
training['ora_valore'] = pd.to_datetime(training['date_time']).dt.hour
training['ora_valore'].unique()
df_corr= training.iloc[ :,list(range(1, 18)) + [-2,-1]]
df_corr.corr()

In [ ]:
sns.heatmap(df_corr.corr(), annot=False, cmap='GnBu')

# 🤖 Creazione del Modello di Regressione

Adesso siamo pronti a costruire un **modello predittivo** che ci permetta di stimare la **qualità di un batch** (unità oraria di produzione) sulla base:

- dei dati raccolti minuto per minuto durante il processo di tostatura
- delle variabili statiche iniziali (umidità e altezza strato)
- delle nuove feature aggregate, come la **temperatura media per camera**

Il nostro obiettivo è all

## 🛠️ Feature Engineering: media della temperatura per camera

Nel contesto di questo problema, ogni camera della macchina è dotata di 3 sensori di temperatura.

Dall’analisi preliminare, abbiamo osservato che:
- I sensori di ciascuna camera forniscono letture **molto simili**
- Le variazioni tra sensori della stessa camera sono **minime**

👉 In questo caso, ha più senso **riassumere il comportamento di ogni camera** calcolando la **temperatura media dei suoi 3 sensori**.  
In questo modo:
- Riduciamo il numero di feature
- Aumentiamo la robustezza del modello
- Rappresentiamo meglio il funzionamento termico medio della camera

### ✅ Obiettivo:
Creare 5 nuove variabili:
- `temp_camera_1_mean`, `temp_camera_2_mean`, ..., `temp_camera_5_mean`

Queste nuove feature saranno usate per predire la qualità finale del prodotto.

In [ ]:
temp_camera_1_mean, temp_camera_2_mean, temp_camera_3_mean = training.iloc[:, 1:4].mean(axis=1), training.iloc[:, 4:7].mean(axis=1), training.iloc[:, 7:10].mean(axis=1)
temp_camera_4_mean, temp_camera_5_mean = training.iloc[:, 10:13].mean(axis=1), training.iloc[:, 13:16].mean(axis=1)

In [ ]:
new_df_train = pd.DataFrame({
    'temp_camera_1_mean': temp_camera_1_mean,
    'temp_camera_2_mean': temp_camera_2_mean,
    'temp_camera_3_mean': temp_camera_3_mean,
    'temp_camera_4_mean': temp_camera_4_mean,
    'temp_camera_5_mean': temp_camera_5_mean,
    'H_data': training['H_data'],
    'quality': training['quality']
})
sns.heatmap(new_df_train.corr(), annot=False, cmap='GnBu')

## Adattamento del modello di regressione lineare

In questa fase abbiamo addestrato un modello di regressione lineare con l'obiettivo di predire la qualità del prodotto (`quality`). Il modello mostra una buona capacità predittiva generale, con un coefficiente di determinazione \( R^2 \) di circa 0.76 sia sul training che sul test set, e un errore medio assoluto (MAE) contenuto rispetto all’intervallo dei valori della variabile target.

Tuttavia, l’analisi dei residui ha evidenziato la presenza di errori particolarmente elevati in alcuni casi (fino a −300), suggerendo che il modello lineare potrebbe non essere in grado di catturare appieno la complessità del fenomeno. Questo comportamento anomalo indica la possibile presenza di outlier o relazioni non lineari tra le variabili.

Per migliorare la qualità delle predizioni, sarà opportuno valutare modelli più flessibili (ad esempio Random Forest ) e analizzare più approfonditamente la natura degli outlier nei residui.


In [ ]:
#adesso siamo pronti per fare una regressione lo scopo principlale è quello di predirre quality
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error


X_train, X_test, y_train, y_test = train_test_split(new_df_train.drop('quality', axis=1), new_df_train['quality'], test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

score_training = model.score(X_train, y_train)
score_test = model.score(X_test, y_test)

print(f'Score training: {score_training}')
print(f'Score test: {score_test}')

In [ ]:
residuals = y_test - y_pred
sns.histplot(residuals)
min(residuals),max(residuals)

In [ ]:
df = pd.DataFrame({
    'y_test': y_test,
    'y_pred': y_pred,
    'res': residuals
})
sum(abs(df['res'])>300)

In [ ]:
# val_hour=training['date_hour'].unique()
# val_hour[0]

In [ ]:
# #split columns training for minute
# part_train = training[training['date_hour']==val_hour[0]].T
# part_train.columns +=1
# part_train.columns
# columns = part_train.index[range(1,16)]

# new_name_columns = []

# for _ in columns :
#     for i in range(1,61):
#         new_name_columns.append(f"{_}_{i}")
# flattened_array = part_train.T[columns].to_numpy().flatten()

In [ ]:
# new_training = pd.DataFrame(columns=new_name_columns)
# new_training

In [ ]:
# for hour in val_hour:
#     part_train = training[training['date_hour'] == hour]

#     # Assicuriamoci che 'columns' esistano nel sotto-dataset
#     flattened_array = part_train[columns].to_numpy().flatten()

#     # Aggiungiamo come una nuova riga nel DataFrame
#     new_training = pd.concat(
#         [new_training, pd.DataFrame([flattened_array],  columns=new_name_columns)],
#         ignore_index=True
#     )